In [ ]:
# Import Spark streaming and Kafka
import os  
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.2 pyspark-shell'  
from pyspark import SparkContext
from pyspark.streaming import StreamingContext  
from pyspark.streaming.kafka import KafkaUtils  
from datetime import datetime

# Define micro batch window and sliding window
batch_interval = 1
window_length = 12 * batch_interval
frequency = 6 * batch_interval

# Define spark context
sc = SparkContext(appName="CTR")
sc.setLogLevel("WARN")
ssc = StreamingContext(sc, batch_interval)

# Create RDD
directKafkaStream = KafkaUtils.createDirectStream(ssc, ['topic_click'], {"metadata.broker.list": 'localhost:9092'})

In [ ]:
lines = directKafkaStream.map(lambda x: x[1])

In [ ]:
# Define function for MapReduce
import re
def mk_int(s):
    s = s.strip()
    return int(s) if s else 0
def tuple_sum(a, b):
    return [a[0]+b[0], a[1]+b[1]]
def tuple_minus(a, b):
    return [a[0]-b[0], a[1]-b[1]]

In [ ]:
# Insert stream to MySQL database
def insert_into_mysql(group_id, clicks, total):
    import MySQLdb
    conn = MySQLdb.connect(host = "localhost", user = "uid", passwd = "pwd", db = "db")
    
    # Control Start and End of ABtest
    try:
        x = conn.cursor()
        x.execute("select yn from abonoff;")
        abonoff = 0
        for row in x:
            abonoff = int(row[0])
        if abonoff == 1:
            if group_id == 1:
                x.execute("""INSERT INTO ctr_4 VALUES (%s,%s,%s,%s,%s,%s,%s)""",(datetime.now(), "test", clicks, total, " ", " ", datetime.now()))
            else:
                x.execute("""INSERT INTO ctr_4 VALUES (%s,%s,%s,%s,%s,%s,%s)""",(datetime.now(), "control", clicks, total, " ", " ", datetime.now()))
        else:
            x.execute("""INSERT INTO ctr_4 VALUES (%s,%s,%s,%s,%s,%s,%s)""",(datetime.now(), "control", clicks, total, " ", " ", datetime.now()))
        conn.commit()
        return (str(group_id) + " " + str(abonoff), 100.0 * clicks / total)
    except Exception, e:
        conn.rollback()
        return (str(e), 100.0 * clicks / total)
    conn.close()

In [ ]:
user_info = sc.textFile("s3n://s3bucketnm/UserInfo.tsv")
user_info = user_info.map(lambda line: line.split("\t")[:])
type(user_info)

In [ ]:
# Spark streaming reduce by key and window
ctr = lines.map(lambda line: line.split("\t")) \
    .map(lambda click: (click[0], mk_int(click[-1]))) \
    .reduceByKey(lambda a, b: a+b) \
    .map(lambda kvpair: (mk_int(kvpair[0])%2, (kvpair[1], 1))) \
    .reduceByKeyAndWindow(lambda a, b: tuple_sum(a, b), None, window_length, frequency) \
    .flatMap(lambda row: insert_into_mysql(row[0], row[1][0], row[1][1]) )
    
ctr.pprint(10)

In [ ]:
ssc.start()  
ssc.awaitTermination()

In [ ]:
ssc.stop()
sc.stop()